In [ ]:
from pyhive import hive
from common_vars import *

In [ ]:
#USE WITH CAUTION

#DELETE and recreate the data and checkpoint directories in hdfs 



def reset_dir(hdfs_path):
    if fs.exists(hdfs_path):
        fs.delete(hdfs_path,recursive=True)
        print("directory deleted: " + hdfs_path)
    else:
        print("no such dir: " + hdfs_path)

    fs.mkdir(hdfs_path + '/sparkcheckpoint')    
    print("directory created: " + fs.ls(hdfs_path)[0])
    fs.chmod(hdfs_path + '/sparkcheckpoint', 0o777)   
    fs.chmod(hdfs_path, 0o777)    

In [ ]:
#queries must be list of strings ["select from...", "insert into..." ]
def run_queries(queries):
    with hive_cnx.cursor() as cursor:   
        for q in queries:
            cursor.execute(q)    
        if cursor.poll().hasResultSet:
            return cursor.fetchall()

In [ ]:
reset_dir(hdfs_archive_path)

In [ ]:
reset_dir(hdfs_hive_staging)

## Create  HIVE Database

In [ ]:
hive_cnx = hive.Connection(
  host = hdfs_host, 
  port = hive_port, 
  username = hive_username,
  password = hive_password,
  auth = hive_mode)

In [ ]:
# q = f"drop database if exists {hive_database} cascade "

# with hive_cnx.cursor() as cursor:
#     cursor.execute(q)

In [ ]:
reset_dir(f"{hdfs_hive_warehouse}/{hive_database}.db")

In [ ]:
run_queries([f"create database if not exists {hive_database}"])

In [ ]:
hive_cnx = hive.Connection(
  host = hdfs_host, 
  port = hive_port, 
  username = hive_username,
  password = hive_password,
  auth = hive_mode,
  database=hive_database)

## Create tweets table

In [ ]:
table = "tweets"
colNames = tweet_keys.copy()
colNames += ["Sentiment","n_words"]

colTypes = [t().simpleString() for t in tweet_types]
colTypes += ['int','int']


In [ ]:
list(zip(colNames,colTypes))

In [ ]:
colCreate = ', '.join([n + ' ' + t for n,t in zip(colNames, colTypes)])
colCreate

In [ ]:
reset_dir(hdfs_hive_tweets)

In [ ]:
q =  f"CREATE EXTERNAL TABLE IF NOT EXISTS {table} ({colCreate}) \
partitioned by (key string)\n \
STORED AS PARQUET \
LOCATION  '{hdfs_hive_tweets}'"

# f"LOAD DATA INPATH '{hdfs_hive_staging}/*.parquet' \
# INTO TABLE tweets \
# partition by key "
print(q)

#TBLPROPERTIES ('avro.schema.url'='hdfs://$tempdir/avroSchema/$tbl.avsc') ;" 

In [ ]:
run_queries([q])

## Create Users Table

In [ ]:
table = "users"
staging = "users" # must actually be "users_staging", but the aggregation strategy does not work (see below) 
colNames = ['last_tweet_at'] + user_keys[:-1]
colTypes = [t().simpleString() for t in [StringType] + user_types][:-1]

In [ ]:
colCreate = ', '.join([n + ' ' + t for n,t in zip(colNames, colTypes)])
colCreate

In [ ]:
reset_dir(hdfs_hive_users)

In [ ]:
reset_dir(hdfs_hive_users_staging)

In [ ]:
q1 =  [f"CREATE EXTERNAL TABLE IF NOT EXISTS {table} ({colCreate}) \
partitioned by ({partitionCol} string) \
STORED AS PARQUET \
LOCATION '{hdfs_hive_users}'"]

# f"LOAD DATA INPATH '{hdfs_hive_staging}/*.parquet' \
# INTO TABLE tweets \
# partition by key "
print(q1)

#TBLPROPERTIES ('avro.schema.url'='hdfs://$tempdir/avroSchema/$tbl.avsc') ;" 

In [ ]:
q1 +=  [f"CREATE TABLE IF NOT EXISTS {staging} ({colCreate}) \
partitioned by ({partitionCol} string) \
STORED AS PARQUET \
LOCATION '{hdfs_hive_users_staging}'"]

# f"LOAD DATA INPATH '{hdfs_hive_staging}/*.parquet' \
# INTO TABLE tweets \
# partition by key "
print(q1)

#TBLPROPERTIES ('avro.schema.url'='hdfs://$tempdir/avroSchema/$tbl.avsc') ;" 

In [ ]:
run_queries(q1)

In [ ]:
from pyspark.sql.types import StructType, StringType, MapType, StructField,\
                              BooleanType, DateType, NumericType, IntegerType,\
                              LongType, TimestampType, FloatType, ArrayType
import pyarrow as pa


topic = 'TweeterArchive'
partitionCol = "created_ym"
Keywords = 'Israel'


hdfs_host = 'localhost'
hdfs_port = 9870
hive_port = 10000
hive_username = 'hdfs'
hive_password = 'naya'
hive_database = 'twitter'
hive_mode = 'CUSTOM'


fs = pa.hdfs.connect(
    host=hdfs_host, 
    port=8020, 
    user=hive_username, 
    kerb_ticket=None, 
    driver='libhdfs', 
    extra_conf=None)


event_fields = [ 'id', 'text','created_at', 'geo', 'coordinates', 'place',
                 'quote_count', 'reply_count', 'retweet_count', 'favorite_count' ]

tweet_keys =  event_fields + ['user_id', 'user_followers' ]

tweet_types = [LongType, StringType, TimestampType, StringType, StringType, StringType, 
               IntegerType, IntegerType, IntegerType, IntegerType, LongType, IntegerType]

# tweet_types  = [StringType]* 11 


user_fields = ['id', 'name', 'screen_name','created_at', 'location', 'url',
                         'protected', 'verified', 'followers_count', 'friends_count',
                         'listed_count', 'favourites_count', 'statuses_count', 'withheld_in_countries']

user_keys = user_fields + [partitionCol]

user_types = [LongType, StringType, StringType, TimestampType, StringType, StringType, 
              BooleanType, BooleanType, IntegerType, IntegerType, 
              IntegerType, IntegerType, IntegerType, StringType, StringType]

# user_types = [StringType]* 14





In [ ]:
conf.py